In [14]:
!pip3 install pyarrow

  Using cached pyarrow-1.0.1-cp38-cp38-macosx_10_9_x86_64.whl (11.0 MB)


In [15]:
import os
from pathlib import Path
import geopandas as c
from pyspark.sql import SparkSession
import pandas
from pyspark.sql.types import IntegerType

In [16]:
raw_file_path = "/Users/kzmain/LSDE/data/foil/cln/2010/1.gz.parquet"

In [17]:
def get_file_list(_path):
    _parquet_file_list = []
    for _root, _dirs, _files in os.walk(_path, topdown=False):
        for _name in _files:
            _file_name = os.path.join(_root, _name)
            if Path(_file_name).suffix == '.parquet':
                _parquet_file_list.append(_file_name)
    return _parquet_file_list

In [18]:
def read_from_parquet(_file_list):
    _full_df = pandas.concat(pandas.read_parquet(_parquet_file) for _parquet_file in _file_list)
    _pick_df = _full_df[["pickup_longitude", "pickup_latitude"]]
    _pick_df = _pick_df.drop_duplicates()
    _drop_df = _full_df[["dropoff_longitude", "dropoff_latitude"]]
    _drop_df = _drop_df.drop_duplicates()
    return _pick_df, _drop_df

In [19]:
parquet_file_list = get_file_list(raw_file_path)
pick_df, drop_df = read_from_parquet(parquet_file_list)

In [20]:
display(pick_df)

,pickup_longitude,pickup_latitude
0,-73.9652,40.7910
1,-74.0149,40.7096
2,-73.9595,40.7090
3,-74.0085,40.7350
4,-73.9723,40.7496
...,...,...
26531,-73.9552,40.7758
26546,-73.7782,40.6443
26549,-73.9012,40.7498
26560,-73.8242,40.6651


In [25]:
pick_gdf = gpd.GeoDataFrame(pick_df, geometry=gpd.points_from_xy(pick_df.pickup_longitude, pick_df.pickup_latitude))

In [29]:
import osmnx as ox
map_dir = map_dbfs= os.path.join("../data", "nyc/map")
nyc_mph = os.path.join(map_dir, "NYC.mph")
nyc_map = ox.load_graphml(nyc_mph)

In [37]:
edges = ox.get_nearest_edges(nyc_map, pick_df["pickup_longitude"], pick_df["pickup_latitude"], method='kdtree', dist=0.0001)

In [38]:
len(edges)

504528